In [0]:
import pandas as pd
import numpy as np

In [0]:
spark.catalog.setCurrentDatabase("default")

In [0]:
#SMA Cross Over 

# Converting Table to Dataframe
df=spark.table('sw_hot_table').toPandas()
df=df.sort_values("Date").reset_index(drop=True)

# Creating SMA
df['SMA_50']=df['Close_Price'].rolling(window=50).mean()
df['SMA_200']=df['Close_Price'].rolling(window=200).mean()

#SMA Strategy
df['SMA_Signal']=0
df.loc[(df['SMA_50']>df['SMA_200']) & (df['SMA_50'].shift(1)<=df['SMA_200'].shift(1)),'SMA_Signal']=1

df['SMA_Position']=np.where(df['SMA_50']>df['SMA_200'],1,0)

df['Daily_Returns']=df['Close_Price'].pct_change()
df['Cum_Daily_Returns']=((df['Daily_Returns']/100+1).cumprod()-1)*100

df['SMA_Strategy']=(df['Daily_Returns']*df['SMA_Position'])
df['Cum_SMA_Strategy']=((df['SMA_Strategy']+1).cumprod()-1)

#df['Missed_SMA_Strategy_Pct']=np.where(df['SMA_Position']==0,df['Daily_Returns'],0)
#df['Cum_Missed_SMA_Strategy']=(df['Missed_SMA_Strategy_Pct']+1).cumprod()
#df['Missed_SMA_Strategy_Pct'].plot()

In [0]:
#EMA Strategy

df['EMA_50']=df['Close_Price'].ewm(span=50, adjust=False, min_periods=50).mean()
df['EMA_200']=df['Close_Price'].ewm(span=200,adjust=False,min_periods=200).mean()

df['SMA_Signal']=0
df.loc[(df['SMA_50']>df['SMA_200']) & (df['SMA_50'].shift(1)<=df['SMA_200'].shift(1)),'SMA_Signal']=1

df['EMA_Position']=np.where(df['EMA_50']>df['EMA_200'],1,0)
df['EMA_Position_Returns']=df['Daily_Returns']*df['EMA_Position']

df['Cum_EMA_Strategy']=((df['EMA_Position_Returns']+1).cumprod()-1)*100
display(df['Cum_EMA_Strategy'])

In [0]:
df['TR_Pct']=(df['Close_Price'].diff()/df['Close_Price'].shift(1)).abs()*100
df['ATR_Pct']=df['TR_Pct'].rolling(14).mean()
df['ATR_SMA_Position']=0
threshold_median=df['ATR_Pct'].median()
df['ATR_SMA_Position']=df.loc[(df['SMA_50']>df['SMA_200'])&(df['ATR_Pct']>threshold_median),'ATR_SMA_Position']=1

df['ATR_SMA_Strategy']=df['Daily_Returns']*df['ATR_SMA_Position']/100
df['Cum_ATR_SMA_Strategy']=((df['ATR_SMA_Strategy']+1).cumprod()-1)*100

In [0]:
#Bollinger Band Strategy

df['Middle_Band']=df['Close_Price'].rolling(window=20).mean()
#df['Upper_Band']=df['Middle_Band']+(2*(df['Middle_Band'].rolling(window=20).std()))
df['Lower_Band']=df['Middle_Band']-(2*df['Middle_Band'].rolling(window=20).std())

In [0]:
#2. Momentum Entry Strategy
df['BB_Signal']=0
df.loc[ (df['Close_Price']>df['Lower_Band']) & (df['Close_Price'].shift(1)<=df['Lower_Band'].shift(1)),"BB_Signal"]=1

df['BB_Position']=np.where(df['Close_Price']>df['Lower_Band'],1,0)

df['BB_Strategy']=(df['BB_Position']*df['Daily_Returns']/100)
df['Cum_BB_Strategy']=((df['BB_Strategy']+1).cumprod()-1)*100

df['Cum_BB_Strategy'].plot() #Almost double the return than normal returns !!

In [0]:
df['ATR_BB_Position'] = 0
df.loc[(df['Close_Price'] > df['Lower_Band']) & (df['ATR_Pct'] > threshold_median), 'ATR_BB_Position'] = 1

df['ATR_BB_Strategy']=(df['ATR_BB_Position']*df['Daily_Returns']/100)
df['Cum_ATR_BB_Strategy']=((df['ATR_BB_Strategy']+1).cumprod()-1)*100

In [0]:
#RSI
delta=df['Close_Price'].diff()
gain=(delta.where(delta>0,0)).rolling(14).mean()
loss=(-delta.where(delta<0,0)).rolling(14).mean()
rs=gain/loss
df['RSI']=100-(100/(1+rs))

In [0]:
#MACD
def MACD(data,fast_period,slow_period,signal_line):
    EMA_Fast=data['Close_Price'].ewm(span=fast_period,adjust=False).mean()
    EMA_Slow=data['Close_Price'].ewm(span=slow_period,adjust=False).mean()
    MACD=EMA_Fast-EMA_Slow
    MACD_Signal=MACD.ewm(span=signal_line,adjust=False).mean()
    return MACD,MACD_Signal

df['Monthly_MACD'], df['Monthly_MACD_Signal'] = MACD(df, 22, 12, 6)
# Bi-Monthly MACD
df['Bi_Monthly_MACD'], df['Bi_Monthly_MACD_Signal'] = MACD(df, 39, 19, 9)
#display(df)


In [0]:
#Monthly MACD + RSI (<70)
df['Monthly_MACD_Position']=np.where((df['Monthly_MACD']>df['Monthly_MACD_Signal']) & (df['RSI']<70),1,0)
df['Monthly_MACD_Strategy']=df['Daily_Returns']*df['Monthly_MACD_Position'].shift(1)
df['Cumulative_Monthly_MACD_Strategy']=((1+df['Monthly_MACD_Strategy']).cumprod()-1)*100

#Bi_Monthly MACD + RSI (<70)
df['Bi_Monthly_MACD_Position']=np.where((df['Bi_Monthly_MACD']>df['Bi_Monthly_MACD_Signal']) & (df['RSI']<70),1,0)
df['Bi_Monthly_MACD_Strategy']=df['Daily_Returns']*df['Bi_Monthly_MACD_Position'].shift(1)
df['Cumulative_Bi_Monthly_MACD_Strategy']=((1+df['Bi_Monthly_MACD_Strategy']).cumprod()-1)*100

#Only RSI (<70)
df['RSI_Position']=np.where(df['RSI']<70,1,0)
df['RSI_Strategy']=df['Daily_Returns']*df['RSI_Position'].shift(1)
df['Cumulative_RSI_Strategy']=((1+df['RSI_Strategy']).cumprod()-1)*100

In [0]:
#spark_df=spark.createDataFrame(df)
#spark_df.write.mode("overwrite").option("mergeSchema","true").saveAsTable("_sw_strategy")

In [0]:
%sql
--with combined_table as (
--select Date, Close_Price 
--from sw_Cold_table sct
--where Year>2021
--union all
--select Date, Close_Price
--from sw_Hot_table sht
--where Year<2026
--)
--select percentile_approx(return_pct,0.01) as VaR_99
--from (
--select round((Close_Price-lag(Close_Price) over (order by Date))*100/lag(Close_Price) over (order by Date),2) 
--as return_pct
--from combined_table 
--)
--order by Date;

In [0]:
#from pyspark.sql import functions as F

#strategy_sw=spark.createDataFrame(df)
#strategy_sw=strategy_sw.withColumn('Year',F.year('Date')).withColumn('Month',F.month('Date'))
#(
 #   strategy_sw.write.format("delta").mode("overwrite").option("overwriteSchema","true").partitionBy("year","month").saveAsTable("_sw_strategy")
#)

In [0]:
%sql

select min(Date) as least_date,max(Date) as max_date
 from _sw_strategy;

In [0]:
%sql
describe _sw_strategy

In [0]:
#print(df.columns.tolist())